In [1]:
#Case Study: Log Parsing Domain: Telecom
#A telecom software provider is building an application to monitor different telecom components in the production environment. For monitoring purpose, the #application relies on log files by parsing the log files and looking for potential warning or exceptions in the logs and reporting them.
#The Dataset contains the log files from different components used in the overall telecom application.

#Tasks:
#The volume of data is quite large. As part of the R&D team, you are building a solution
#on spark to load and parse the multiple log files and then arranging the error and by the timestamp.

In [2]:
#Load file as a text file in spark

data='/FileStore/tables/access_1_clean-acc4c.log'

RDD=sc.textFile(data)

In [3]:
RDD.count()

Out[2]: 50000

In [4]:
#Find out how many 404 HTTP codes are in access logs.

import sys
import os

log_file_path = 'dbfs:/' + os.path.join('databricks-datasets', 'cs100', 'lab2', 'data-001', 'apache.access.log.PROJECT')


In [5]:
base_df = sqlContext.read.text(log_file_path)
# Let's look at the schema
base_df.printSchema()

root
-- value: string (nullable = true)

In [6]:
base_df.count()

Out[9]: 1043177

In [7]:
#(2b) Parsing the log file

from pyspark.sql.functions import split, regexp_extract
split_df = base_df.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                          regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('timestamp'),
                          regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                          regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                          regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('content_size'))
split_df.show(truncate=False)

#remotehost rfc931 authuser [date] “request” status bytes
#remotehost	Remote hostname (or IP number if DNS hostname is not available).
#rfc931	The remote logname of the user. We don’t really care about this field.
#authuser	The username of the remote user, as authenticated by the HTTP server.
#[date]	The date and time of the request.
#“request”	The request, exactly as it came from the browser or client.
#status	The HTTP status code the server sent back to the client.
#bytes	The number of bytes (Content-Length) transferred to the client.

+----------------------------+--------------------------+---------------------------------------------------+------+------------+
host |timestamp |path |status|content_size|
+----------------------------+--------------------------+---------------------------------------------------+------+------------+
in24.inetnebr.com |01/Aug/1995:00:00:01 -0400|/shuttle/missions/sts-68/news/sts-68-mcc-05.txt |200 |1839 |
uplherc.upl.com |01/Aug/1995:00:00:07 -0400|/ |304 |0 |
uplherc.upl.com |01/Aug/1995:00:00:08 -0400|/images/ksclogo-medium.gif |304 |0 |
uplherc.upl.com |01/Aug/1995:00:00:08 -0400|/images/MOSAIC-logosmall.gif |304 |0 |
uplherc.upl.com |01/Aug/1995:00:00:08 -0400|/images/USA-logosmall.gif |304 |0 |
ix-esc-ca2-07.ix.netcom.com |01/Aug/1995:00:00:09 -0400|/images/launch-logo.gif |200 |1713 |
uplherc.upl.com |01/Aug/1995:00:00:10 -0400|/images/WORLD-logosmall.gif |304 |0 |
slppp6.intermind.net |01/Aug/1995:00:00:10 -0400|/history/skylab/skylab.html |200 |1687 |
piweba4y.prodigy.com |01/Aug/1995:00:00:10 -0400|/images/launchmedium.gif |200 |11853 |
slppp6.intermind.net |01/Aug/1995:00:00:11 -0400|/history/skylab/skylab-small.gif |200 |9202 |
slppp6.intermind.net |01/Aug/1995:00:00:12 -0400|/images/ksclogosmall.gif |200 |3635 |
ix-esc-ca2-07.ix.netcom.com |01/Aug/1995:00:00:12 -0400|/history/apollo/images/apollo-logo1.gif |200 |1173 |
slppp6.intermind.net |01/Aug/1995:00:00:13 -0400|/history/apollo/images/apollo-logo.gif |200 |3047 |
uplherc.upl.com |01/Aug/1995:00:00:14 -0400|/images/NASA-logosmall.gif |304 |0 |
133.43.96.45 |01/Aug/1995:00:00:16 -0400|/shuttle/missions/sts-69/mission-sts-69.html |200 |10566 |
kgtyk4.kj.yamagata-u.ac.jp |01/Aug/1995:00:00:17 -0400|/ |200 |7280 |
kgtyk4.kj.yamagata-u.ac.jp |01/Aug/1995:00:00:18 -0400|/images/ksclogo-medium.gif |200 |5866 |
d0ucr6.fnal.gov |01/Aug/1995:00:00:19 -0400|/history/apollo/apollo-16/apollo-16.html |200 |2743 |
ix-esc-ca2-07.ix.netcom.com |01/Aug/1995:00:00:19 -0400|/shuttle/resources/orbiters/discovery.html |200 |6849 |
d0ucr6.fnal.gov |01/Aug/1995:00:00:20 -0400|/history/apollo/apollo-16/apollo-16-patch-small.gif|200 |14897 |
+----------------------------+--------------------------+---------------------------------------------------+------+------------+
only showing top 20 rows

In [8]:
# Data Cleaning

#base_df.filter(base_df['value'].isNull()).count()

bad_rows_df = split_df.filter(split_df['host'].isNull() |
                              split_df['timestamp'].isNull() |
                              split_df['path'].isNull() |
                              split_df['status'].isNull() |
                             split_df['content_size'].isNull())
bad_rows_df.count()

Out[12]: 8756

In [9]:
from pyspark.sql.functions import col, sum

def count_null(col_name):
  return sum(col(col_name).isNull().cast('integer')).alias(col_name)


In [10]:
# Build up a list of column expressions, one per column.
#
# This could be done in one line with a Python list comprehension, but we're keeping
# it simple for those who don't know Python very well.
exprs = []
for col_name in split_df.columns:
  exprs.append(count_null(col_name))

# Run the aggregation. The *exprs converts the list of expressions into
# variable function arguments.
split_df.agg(*exprs).show()


+----+---------+----+------+------------+
host|timestamp|path|status|content_size|
+----+---------+----+------+------------+
 0| 0| 0| 0| 8756|
+----+---------+----+------+------------+

In [11]:
bad_content_size_df = base_df.filter(~ base_df['value'].rlike(r'\d+$'))
bad_content_size_df.count()


Out[15]: 8756

In [12]:
from pyspark.sql.functions import lit, concat
bad_content_size_df.select(concat(bad_content_size_df['value'], lit('*'))).show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------+
concat(value, *) |
+----------------------------------------------------------------------------------------------------------------------------+
gw1.att.com - - [01/Aug/1995:00:03:53 -0400] "GET /shuttle/missions/sts-73/news HTTP/1.0" 302 -* |
js002.cc.utsunomiya-u.ac.jp - - [01/Aug/1995:00:07:33 -0400] "GET /shuttle/resources/orbiters/discovery.gif HTTP/1.0" 404 -*|
tia1.eskimo.com - - [01/Aug/1995:00:28:41 -0400] "GET /pub/winvn/release.txt HTTP/1.0" 404 -* |
itws.info.eng.niigata-u.ac.jp - - [01/Aug/1995:00:38:01 -0400] "GET /ksc.html/facts/about_ksc.html HTTP/1.0" 403 -* |
grimnet23.idirect.com - - [01/Aug/1995:00:50:12 -0400] "GET /www/software/winvn/winvn.html HTTP/1.0" 404 -* |
miriworld.its.unimelb.edu.au - - [01/Aug/1995:01:04:54 -0400] "GET /history/history.htm HTTP/1.0" 404 -* |
ras38.srv.net - - [01/Aug/1995:01:05:14 -0400] "GET /elv/DELTA/uncons.htm HTTP/1.0" 404 -* |
cs1-06.leh.ptd.net - - [01/Aug/1995:01:17:38 -0400] "GET /sts-71/launch/" 404 -* |
www-b2.proxy.aol.com - - [01/Aug/1995:01:22:07 -0400] "GET /shuttle/countdown HTTP/1.0" 302 -* |
maui56.maui.net - - [01/Aug/1995:01:31:56 -0400] "GET /shuttle HTTP/1.0" 302 -* |
dialip-24.athenet.net - - [01/Aug/1995:01:33:02 -0400] "GET /history/apollo/apollo-13.html HTTP/1.0" 404 -* |
h96-158.ccnet.com - - [01/Aug/1995:01:35:50 -0400] "GET /history/apollo/a-001/a-001-patch-small.gif HTTP/1.0" 404 -* |
h96-158.ccnet.com - - [01/Aug/1995:01:36:23 -0400] "GET /history/apollo/a-001/movies/ HTTP/1.0" 404 -* |
h96-158.ccnet.com - - [01/Aug/1995:01:36:30 -0400] "GET /history/apollo/a-001/a-001-patch-small.gif HTTP/1.0" 404 -* |
h96-158.ccnet.com - - [01/Aug/1995:01:36:38 -0400] "GET /history/apollo/a-001/movies/ HTTP/1.0" 404 -* |
h96-158.ccnet.com - - [01/Aug/1995:01:36:42 -0400] "GET /history/apollo/a-001/a-001-patch-small.gif HTTP/1.0" 404 -* |
h96-158.ccnet.com - - [01/Aug/1995:01:36:44 -0400] "GET /history/apollo/a-001/images/ HTTP/1.0" 404 -* |
h96-158.ccnet.com - - [01/Aug/1995:01:36:47 -0400] "GET /history/apollo/a-001/a-001-patch-small.gif HTTP/1.0" 404 -* |
h96-158.ccnet.com - - [01/Aug/1995:01:37:04 -0400] "GET /history/apollo/a-004/a-004-patch-small.gif HTTP/1.0" 404 -* |
h96-158.ccnet.com - - [01/Aug/1995:01:37:05 -0400] "GET /history/apollo/a-004/movies/ HTTP/1.0" 404 -* |
+----------------------------------------------------------------------------------------------------------------------------+
only showing top 20 rows

In [13]:
#Fix the rows with null content_size

# Replace all null content_size values with 0.
cleaned_df = split_df.na.fill({'content_size': 0})

# Ensure that there are no nulls left.
exprs = []
for col_name in cleaned_df.columns:
  exprs.append(count_null(col_name))

cleaned_df.agg(*exprs).show()

+----+---------+----+------+------------+
host|timestamp|path|status|content_size|
+----+---------+----+------+------------+
 0| 0| 0| 0| 0|
+----+---------+----+------+------------+

In [14]:
#Parsing the timestamp.

month_map = {
  'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
  'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12
}

def parse_clf_time(s):
    """ Convert Common Log time format into a Python datetime object
    Args:
        s (str): date and time in Apache time format [dd/mmm/yyyy:hh:mm:ss (+/-)zzzz]
    Returns:
        a string suitable for passing to CAST('timestamp')
    """
    # NOTE: We're ignoring time zone here. In a production application, you'd want to handle that.
    return "{0:04d}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}".format(
      int(s[7:11]),
      month_map[s[3:6]],
      int(s[0:2]),
      int(s[12:14]),
      int(s[15:17]),
      int(s[18:20])
    )

u_parse_time = udf(parse_clf_time)

logs_df = cleaned_df.select('*', u_parse_time(split_df['timestamp']).cast('timestamp').alias('time')).drop('timestamp')
total_log_entries = logs_df.count()

logs_df.printSchema()

display(logs_df)


host,path,status,content_size,time
in24.inetnebr.com,/shuttle/missions/sts-68/news/sts-68-mcc-05.txt,200,1839,1995-08-01T00:00:01.000+0000
uplherc.upl.com,/,304,0,1995-08-01T00:00:07.000+0000
uplherc.upl.com,/images/ksclogo-medium.gif,304,0,1995-08-01T00:00:08.000+0000
uplherc.upl.com,/images/MOSAIC-logosmall.gif,304,0,1995-08-01T00:00:08.000+0000
uplherc.upl.com,/images/USA-logosmall.gif,304,0,1995-08-01T00:00:08.000+0000
ix-esc-ca2-07.ix.netcom.com,/images/launch-logo.gif,200,1713,1995-08-01T00:00:09.000+0000
uplherc.upl.com,/images/WORLD-logosmall.gif,304,0,1995-08-01T00:00:10.000+0000
slppp6.intermind.net,/history/skylab/skylab.html,200,1687,1995-08-01T00:00:10.000+0000
piweba4y.prodigy.com,/images/launchmedium.gif,200,11853,1995-08-01T00:00:10.000+0000
slppp6.intermind.net,/history/skylab/skylab-small.gif,200,9202,1995-08-01T00:00:11.000+0000


In [15]:
#Let’s cache logs_df. We’re going to be using it quite a bit from here forward.
logs_df.cache()

Out[19]: DataFrame[host: string, path: string, status: int, content_size: int, time: timestamp]

In [16]:
#Example: HTTP Status Analysis

status_to_count_df =(logs_df
                     .groupBy('status')
                     .count()
                     .sort('status')
                     .cache())

status_to_count_length = status_to_count_df.count()
print('Found %d response codes' % status_to_count_length)
status_to_count_df.show()

assert status_to_count_length == 7
assert status_to_count_df.take(100) == [(200, 940847), (302, 16244), (304, 79824), (403, 58), (404, 6185), (500, 2), (501, 17)]

Found 7 response codes
+------+------+
status| count|
+------+------+
 200|940847|
 302| 16244|
 304| 79824|
 403| 58|
 404| 6185|
 500| 2|
 501| 17|
+------+------+

In [17]:
display(status_to_count_df)

status,count
200,940847
302,16244
304,79824
403,58
404,6185
500,2
501,17


In [18]:
from pyspark.sql import functions as sqlFunctions
log_status_to_count_df = status_to_count_df.withColumn('log(count)', sqlFunctions.log(status_to_count_df['count']))

display(log_status_to_count_df)

status,count,log(count)
200,940847,13.75453581236166
302,16244,9.69547888880619
304,79824,11.287579490100818
403,58,4.060443010546419
404,6185,8.729882284826589
500,2,0.6931471805599453
501,17,2.833213344056216


In [19]:
from spark_notebook_helpers import prepareSubplot, np, plt, cm
data = log_status_to_count_df.drop('count').collect()
x, y = zip(*data)
index = np.arange(len(x))
bar_width = 0.7
colorMap = 'Accent'
cmap = cm.get_cmap(colorMap)

fig, ax = prepareSubplot(np.arange(0, 6, 1), np.arange(0, 14, 2))
plt.bar(index, y, width=bar_width, color=cmap(0))
plt.xticks(index + bar_width/2.0, x)
display(fig)

--------------------------------------------------------------------------- 
 ModuleNotFoundError Traceback (most recent call last)
 <command-855836106911359> in <module> 
 ----> 1 from spark_notebook_helpers import prepareSubplot , np , plt , cm
 2 data = log_status_to_count_df . drop ( 'count' ) . collect ( ) 
 3 x , y = zip ( * data ) 
 4 index = np . arange ( len ( x ) ) 
 5 bar_width = 0.7 

 ModuleNotFoundError : No module named 'spark_notebook_helpers'

In [20]:
# Any hosts that has accessed the server more than 10 times.
host_sum_df =(logs_df
              .groupBy('host')
              .count())

host_more_than_10_df = (host_sum_df
                        .filter(host_sum_df['count'] > 10)
                        .select(host_sum_df['host']))

print('Any 20 hosts that have accessed more then 10 times:\n')
host_more_than_10_df.show(truncate=False)


Any 20 hosts that have accessed more then 10 times:

+-------------------------------+
host |
+-------------------------------+
prakinf2.prakinf.tu-ilmenau.de |
alpha2.csd.uwm.edu |
cjc07992.slip.digex.net |
n1377004.ksc.nasa.gov |
163.205.2.134 |
huge.oso.chalmers.se |
163.205.44.27 |
shark.ksc.nasa.gov |
etc5.etechcorp.com |
dd07-029.compuserve.com |
131.182.101.161 |
134.95.100.201 |
vab08.larc.nasa.gov |
ip11.iac.net |
ad11-012.compuserve.com |
ad053.du.pipex.com |
204.184.6.19 |
p8.denver1.dialup.csn.net |
gate2.gdc.com |
alcott.acsu.buffalo.edu |
+-------------------------------+
only showing top 20 rows

In [21]:
unique_host_count = (logs_df.select('host')
                     .distinct()
                     .count())
print ('Unique hosts: {0}'.format(unique_host_count))


Unique hosts: 54507